In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
import exchange_calendars as ecals

In [3]:
import tkinter
import tkinter.font
import tkinter.messagebox as msgbox

In [4]:
csv00 = pd.read_csv('./QQQ00.csv').to_numpy()
csv10 = pd.read_csv('./QQQ10.csv').to_numpy()

In [5]:
csv00_sample = csv00[:,0]
csv00_key = [i.replace(' ','') for i in csv00_sample]
csv00_value = csv00[:,1]
data00 = np.column_stack((csv00_key, csv00_value))
data00 = list(reversed(data00))

In [6]:
csv10_sample = csv10[:,0]
csv10_key = [i.replace(' ','') for i in csv10_sample]
csv10_value = csv10[:, 1]
data10 = np.column_stack((csv10_key, csv10_value))
data10 = list(reversed(data10))

In [7]:
#2005년 1월부터 2023년 12월까지의 QQQ 주식 정보가 들어있는 변수 [날짜, 종가]
data = data00+data10

In [8]:
kr = ecals.get_calendar("XKRX") #한국
NYSE = ecals.get_calendar("NYSE") #뉴욕
#주식시장 개장일 확인 함수
def checkOpenday(date): #문자열 사용
    date = date.strftime("%Y-%m-%d")
    return NYSE.is_session(date)

In [9]:
#다음주 원하는 요일 날짜 구하는 함수, datetime 형식 사용
#day가 0이면 다음주 월요일 날짜를 구함
def onDay(date, day): # date = 현재날짜, day = 0:월 1:화 ~
    days=(day - date.weekday()) % 7
    if days==0:
        days=7
    return date + timedelta(days=days)

In [10]:
#1주일 뒤 가장 처음 주식시장 개장 날짜 구하는 함수
def after1week(date):
    date = datetime.strptime(date, '%Y-%m-%d') #문자열에서 변환
    date = onDay(date, 0) 
    while(True):
        if checkOpenday(date): #주식시장이 개장하는 날짜면 True 반환
            break
        else:
            date = date + timedelta(days=1)
    return date.strftime('%Y-%m-%d') #문자열로 변환

In [11]:
#2주일 뒤 가장 처음 주식 개장 날짜
def after2week(date):
    date = datetime.strptime(date, '%Y-%m-%d') #문자열에서 변환
    date = onDay(date, 0)
    date = onDay(date, 0)
    while(True):
        if checkOpenday(date):
            break
        else:
            date = date + timedelta(days=1)
    return date.strftime('%Y-%m-%d') #문자열로 변환

In [12]:
#한달 뒤 가장 처음 주식 개장 날짜
def after1month(date):
    date = datetime.strptime(date, '%Y-%m-%d') #문자열에서 변환
    month = date.month + 1
    year = date.year
    if month == 13:
        month=1
        year = year+1
    new_date = datetime(year, month, 1)
    
    while(True):
        if checkOpenday(new_date):
            break
        else:
            new_date = new_date + timedelta(days=1)
    return new_date.strftime('%Y-%m-%d') #문자열로 변환

In [14]:
#테스트용
#data2=[["2004-01-08",37.82],["2004-01-09",37.57],["2004-01-12",38.17],["2004-01-13",37.79],["2004-01-14",37.92],["2004-01-15",38.02],["2004-01-16",38.41],["2004-01-20",38.38],["2004-01-21",38.22],["2004-01-22",37.99],["2004-01-23",37.85],["2004-01-26",38.43]]
#print(after1week('2023-12-27'))

In [15]:
def start():
    #입력칸 숫자,공백 확인
    money = entry.get() #매매금액
    if money.isdigit()==False or money=="":
        msgbox.showwarning("경고","매매금액을 정확히 입력해 주세요")
        return
    money = int(money)
    if money <500:
        msgbox.showwarning("경고", "매매금액을 500 이상 입력해주세요")
        return
    
    gijun=0
    if jogeon.get()==0:
        gijun = entry3.get() #수익률조건
        if gijun.isdigit()==False or gijun=="":
            msgbox.showwarning("경고", "수익률 조건을 정확히 입력해 주세요")
            return
    else:
        gijun = entry4.get() #수익금조건
        if gijun.isdigit()==False or gijun=="":
            msgbox.showwarning("경고", "수익금 조건을 정확히 입력해 주세요")
            return
    gijun = int(gijun)
    
    sell_per = entry2.get() #수익실현비율
    if sell_per.isdigit()==False or sell_per=="":
        msgbox.showwarning("경고","수익실현비율을 정확히 입력해 주세요")
        return
    sell_per = int(sell_per)
    ############################
    
    #각종 변수 선언
    buyDate=data[0][0]
    maxStartDate=datetime.strptime(buyDate, '%Y-%m-%d')
    maxEndDate=maxStartDate
    dateCha = 0
    maxCount=0
    maxMoneySum=0
    currentCount=0
    currentMoneySum=0
    average=0
    suik=0
    sell_days=[buyDate]
    sell_suiks=[0]
    during=[]
    for i in data: 
        if i[0]==buyDate: #매수하는 날짜일 때
            n = money//i[1] #매수 가능 개수 구하기
            average = (average*currentCount+i[1]*n)/(currentCount+n)
            currentCount += n #주식 보유수 증가
            if currentCount > maxCount: #주식 보유량 최다일때
                maxCount = currentCount
            currentMoneySum += n*i[1]
            if currentMoneySum > maxMoneySum: #주식 매수금액 최대일때
                maxMoneySum = currentMoneySum
            
            #다음 주식 매수 날짜 구하기
            when = cycle.get() 
            if when==1:
                buyDate=after1week(buyDate)
            elif when==2:
                buyDate=after2week(buyDate)
            elif when==4:
                buyDate=after1month(buyDate)
                
            #a = buyDate +"-"+str(maxCount)+"-"+str(maxMoneySum)+"-"+str(currentCount)+"-"+str(currentMoneySum)+"-"+str(average)
            #msgbox.showinfo("asdf", a)
            
        #매도
        cha = (i[1]-average)*currentCount #평가손익
        sellCount = int(currentCount * sell_per / 100) #매도 개수
        state=0
        if jogeon.get() == 0: #수익률이 조건일때
            cha = cha/currentMoneySum*100
            if cha>=gijun:
                state=1
        elif jogeon.get() == 1: #수익금이 조건일때
            if cha >= gijun:
                state=1
        if state==1: #매도 조건 성립
            suik += (i[1]-average)*sellCount
            sell_suiks.append(suik)
            sell_days.append(i[0])
            currentCount -= sellCount
            currentMoneySum -= average*sellCount
            a=datetime.strptime(i[0], '%Y-%m-%d')
            c=datetime.strptime(sell_days[-2],'%Y-%m-%d')
            cc = (a-c).days
            during.append(cc)
            b=(a-maxEndDate).days
            if b>dateCha:
                dateCha = b
                maxStartDate=maxEndDate
                maxEndDate = a
                
    #abc=str(suik)+"-"+str(currentCount)+"-"+str(currentMoneySum)+"-"+str(dateCha)+"-"+maxStartDate.strftime('%Y-%m-%d')+"-"+maxEndDate.strftime('%Y-%m-%d')
    #abc += "-"+str(maxCount) +"-"+str(maxMoneySum)
    #print(during)
    #print(sell_days)
    #print(sell_suiks)
    #msgbox.showinfo("Asdf", abc)
    ######################################
    
    
    #결과창 띄우기
    during_aver = np.mean(during)
    #print(during_aver)
    
    app = tkinter.Tk()
    app.title("백테스트 결과 확인")
    
    wid=400
    hei=300
    x2 = (screen_width/2) - (wid/2)
    y2 = (screen_height/2) - (hei/2)
    
    app.geometry("%dx%d+%d+%d" % (wid, hei, x2, y2))
    app.resizable(False, False)
    ff = tkinter.font.Font(size=13, weight="bold")
    
    aa = str(money)+"$ 매수, "
    bb="empty"
    cc="empty"
    when = cycle.get()
    if when==1:
        bb="1주마다, "
    elif when==2:
        bb="2주마다, "
    elif when==4:
        bb="1달마다, "
    if jogeon.get()==0:
        cc=str(gijun)+"%"
    else:
        cc=str(gijun)+"$"
    dd=str(sell_per)+"% 매도"
    txt0= bb+aa+"수익 "+cc+" 달성 시 "+dd

    txt1 = "총 수익금 : %d $" % suik
    txt2 = "최다 주식 보유 수량 : %d" % maxCount
    txt3 = "최대 매수 금액 : %d $" % maxMoneySum
    txt4 = "평균 매매 주기 : %d일" % during_aver
    txt5 = "최장 수익 미실현 기간 : %s ~ %s, %d일" % (maxStartDate.strftime('%Y-%m-%d'), maxEndDate.strftime('%Y-%m-%d'), dateCha)
    txt6 = "수익 실현 횟수 : %d" % len(during)
    
    res0 = tkinter.Label(app)
    res0['text']=txt0
    res0.place(x=40, y=40)
    
    res1 = tkinter.Label(app)
    res1['text']=txt1
    res1.place(x=40, y=80)
    
    res2 = tkinter.Label(app)
    res2['text']=txt2
    res2.place(x=40, y=100)
    
    res3 = tkinter.Label(app)
    res3['text']=txt3
    res3.place(x=40, y=120)
    
    res4 = tkinter.Label(app)
    res4['text']=txt4
    res4.place(x=40, y=140)
    
    res5 = tkinter.Label(app)
    res5['text']=txt5
    res5.place(x=40, y=160)
    
    res6 = tkinter.Label(app)
    res6['text']=txt6
    res6.place(x=40, y=180)
    
    
    app.mainloop()

In [16]:
def jogeon_state():
    if jogeon.get() == 0:
        entry4.configure(state="disabled")
        entry3.configure(state="normal")
    elif jogeon.get() == 1:
        entry3.configure(state="disabled")
        entry4.configure(state="normal")
#첫 화면
window=tkinter.Tk()
window.title("QQQ 백테스팅 프로그램")

font=tkinter.font.Font(size=14, weight="bold")
font2=tkinter.font.Font(size=13, weight="bold")
font3=tkinter.font.Font(size=13)

# 크기 설정
width = 640
height = 400

screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()    
x = (screen_width/2) - (width/2)
y = (screen_height/2) - (height/2)

window.geometry("%dx%d+%d+%d" % (width, height, x, y))
window.resizable(False, False)
################################################

#기간 표시
period = tkinter.Label(window, text="2005.01 ~ 2023.12", font=font)
period.place(x=230, y=10)
##################

#매매금액
label2 = tkinter.Label(window, text="매수금액", font=font2)
label2.place(x=80, y=80)
##################

#매매금액 입력칸
entry=tkinter.Entry(window, width=8, font=font2)
entry.insert(0, "1000")
entry.place(x=80, y=105)
dollar = tkinter.Label(window, text="$" ,font=font3)
dollar.place(x=155, y=105)
#################

#수익실현조건
label3 = tkinter.Label(window, text="수익실현 조건", font=font2)
label3.place(x=260, y=80)
#라디오버튼
jogeon = tkinter.IntVar(value="1")
j_rad1 = tkinter.Radiobutton(window, text="수익률", variable = jogeon, value=0, font=font3, command=jogeon_state)
j_rad1.place(x=275, y=120)
entry3=tkinter.Entry(window, width=4, font=font2)
entry3.insert(0, "10")
entry3.configure(state="disabled")
entry3.place(x=285, y=150)
j_label=tkinter.Label(window, text="%", font=font3)
j_label.place(x=330, y=150)

j_rad2 = tkinter.Radiobutton(window, text="수익금", variable = jogeon, value=1, font=font3, command=jogeon_state)
j_rad2.place(x=275, y=200)
entry4=tkinter.Entry(window, width=8, font=font2)
entry4.insert(0, "500")
entry4.place(x=280, y=225)
j_label2=tkinter.Label(window, text="$", font=font3)
j_label2.place(x=360, y=225)
##################

#수익실현 비율
label4 = tkinter.Label(window, text="수익실현 비율", font=font2)
label4.place(x=460, y=80)
#퍼센트 비율
entry2=tkinter.Entry(window, width=4, font=font2)
entry2.insert(0, "50")
entry2.place(x=470, y=105)
per = tkinter.Label(window, text="% 매도", font=font3)
per.place(x=510, y=105)
##################

#매매주기
label5 = tkinter.Label(window, text="매수주기", font=font2)
label5.place(x=80, y=180)
#라디오버튼
cycle = tkinter.IntVar(value="1")
c_rad1 = tkinter.Radiobutton(window, text="1주", variable = cycle, value=1, font=font3)
c_rad1.place(x=90, y=210)

c_rad2 = tkinter.Radiobutton(window, text="2주", variable = cycle, value=2, font=font3)
c_rad2.place(x=90, y=235)

c_rad3 = tkinter.Radiobutton(window, text="1달", variable = cycle, value=4, font=font3)
c_rad3.place(x=90, y=260)
##################

#백테스팅 시작 버튼
end_button = tkinter.Button(window, text="시작", font=font, command=start)
end_button.place(x=290, y=340)
#################

window.mainloop()

